In [12]:
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Dark-
[nltk_data]     Devil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_word_text = "a, an, the, and, or, but, if, then, else, when, where, who, whom, which, that, this, these, those, in, on, at, to, from, by, for, of, with, without, over, under, above, below, between, among, through, throughout, until, while, since, during, within, without, beyond, beside, between, except, but, up, down, in, out, off, above, below, under, too, very, so, such, just, as, both, neither, either, although, because, since, so that, though, unless, until, whether, while, why"

In [23]:
stop_words = stop_word_text.split(",")

In [14]:
#filtering the text
def full_form(text):
    text = text.lower()
    plain = re.sub(r'[<>?\.,!"(\)\/[\]]', '', text)
    plain = plain.replace("don't", "do not")
    plain = plain.replace("won't", "will not")
    plain = plain.replace("haven't", "have not")
    plain = plain.replace("can't", "cannot")
    plain = plain.replace("she's", "she is")
    plain = plain.replace("he's", "he is")
    plain = plain.replace("there're", "there are")
    plain = plain.replace("they'd", "they would")
    plain = plain.replace("\'ll", " will")
    return plain              

In [15]:
#initialize the stemmer and lemmentizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [24]:
#preprocessing  text 
def remove_stop_words(data):
    corpus = []
 
    #split the sentence
    plain = full_form(data)
    sentence = plain.split()
    
    #check and stem the word
    review_processed = [stemmer.stem(word) for word in sentence if not word in stop_words]
    #rebuild the sentence
    review_joint = ' '.join(review_processed)
    #add the sentence into list
    corpus.append(review_joint)
    return corpus

In [17]:
df = pd.read_csv("C:\\Users\\Dark-Devil\\Desktop\\IMDB Dataset.csv")

In [82]:
print(df['review'][103],'\n')
print(df['sentiment'][103])

No, this hilariously horrible 70's made-for-TV horror clinker isn't about a deadly demonically possessed dessert cake. Still, this exceptionally awful, yet undeniably amusing and thus enjoyable cathode ray refuse reaches a breathtaking apex of absolute, unremitting silliness and atrociousness that's quite tasty in a so-execrable-it's-downright-awesome sort of way. Richard Crenna, looking haggard and possibly inebriated, and Yvette Mimieux, who acts as if she never got over the brutal rape she endured in "Jackson County Jail," sluggishly portray a disgustingly nice and respectable suburbanite couple whose quaint, dull, sleepy small town existence gets ripped asunder when the cute German Shepard they take in as the family pet turns out to be some ancient lethal evil spirit. Pretty soon Mimieux and her two repellently cutesy kids Kim Richards and Ike Eisenmann (the psychic alien moppets from the Disney "Witch Mountain" pictures) are worshiping a crude crayon drawing of the nasty, ugly can

In [19]:
sent_length = 200
#vocabular size
voc_size = 5000

In [83]:
text = """
    No, this hilariously horrible 70's made-for-TV horror clinker isn't about a deadly demonically possessed dessert cake. 
    Still, this exceptionally awful, yet undeniably amusing and thus enjoyable cathode ray refuse reaches a 
    breathtaking apex of absolute, unremitting silliness and atrociousness that's quite tasty in a 
    so-execrable-it's-downright-awesome sort of way. Richard Crenna, looking haggard and possibly inebriated, and 
    Yvette Mimieux, who acts as if she never got over the brutal rape she endured in "Jackson County Jail," sluggishly 
    portray a disgustingly nice and respectable suburbanite couple whose quaint, dull, sleepy small town existence gets ripped 
    asunder when the cute German Shepard they take in as the family pet turns out to be some ancient lethal evil spirit. 
    Pretty soon Mimieux and her two repellently cutesy kids Kim Richards and Ike Eisenmann (the psychic alien moppets from 
    the Disney "Witch Mountain" pictures) are worshiping a crude crayon drawing of the nasty, ugly canine entity in the den. 
    Boy, now doesn't that sound really scary and disturbing? Well, scary and disturbing this laughably ludicrous claptrap 
    sure ain't, but it sure is funny, thanks to Curtis ("Night Tide") Harrington's hopelessly weak direction, cartoonish 
    (not so) special effects, an almost painfully risible'n'ridiculous plot, and a game cast that struggles valiantly with 
    the absurd story (besides the leads, both Martine Beswicke and R.G. Armstrong briefly pop up as members of a Satanic 
    cult and Victor Jory has a nice cameo as a helpful Native American shaman). Favorite scene: the malicious Mephestophelion
    mutt puts the whammy on Crenna, practically forcing him to stick his hand into a wildly spinning lawnmower blade. While 
    stuck-up snobby fright film fans may hold their noses at the perfectly putrid stench of this admittedly smelly schlock,
    devout TV trash lovers should deem this endearingly abominable offal the boob tube equivalent to Alpo. 
"""

In [84]:
#preprocessing 
processed_text = remove_stop_words(text)
#one hot encoding
onehot_sent = [one_hot(word, voc_size) for word in processed_text]
#embedding
embedd_docs = pad_sequences(onehot_sent, padding='pre', maxlen=sent_length)
#convert into numpy array
X_sample = np.array(embedd_docs)

In [85]:
processed_text

["no thi hilari horribl 70' made-for-tv horror clinker isn't about deadli demon possess dessert cake still thi except aw yet undeni amus and thu enjoy cathod ray refus reach breathtak apex of absolut unremit silli and atroci that' quit tasti in so-execrable-it's-downright-awesom sort of way richard crenna look haggard and possibl inebri and yvett mimieux who act as if she never got over the brutal rape she endur in jackson counti jail sluggishli portray disgustingli nice and respect suburbanit coupl whose quaint dull sleepi small town exist get rip asund when the cute german shepard they take in as the famili pet turn out to be some ancient lethal evil spirit pretti soon mimieux and her two repel cutesi kid kim richard and ike eisenmann the psychic alien moppet from the disney witch mountain pictur are worship crude crayon draw of the nasti ugli canin entiti in the den boy now doesn't that sound realli scari and disturb well scari and disturb thi laughabl ludicr claptrap sure ain't but

In [88]:
rnn = load_model("./assets/movieSentiAnalysisV-3.h5")

In [86]:
import json
from keras.models import model_from_json

# Load the saved model architecture
with open('./assets/my_model-V3.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Create the Keras model from the loaded architecture
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('./assets/my_model_weights-V3.h5')

In [90]:
sample_prediction = (rnn.predict(X_sample) > 0.5).astype("int32")
sample_prediction

1/1 [==============================] - 0s 16ms/step


array([[0]])